In [9]:
import requests
import csv
from multiprocessing.pool import ThreadPool
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

In [10]:
topic_dict = {
    '01': 'любовная лирика',
    '14': 'гражданская лирика',
    '02': 'пейзажная лирика',
    '08': 'городская лирика',
    '17': 'философская лирика',
    '13': 'мистика и эзотерика',
    '22': 'сонеты, канцоны, рондо',
    '11': 'шуточные стихи',
    '16': 'иронические стихи',
    '34': 'сатирические стихи',
    '41': 'басни',
    '10': 'стихи для детей',
    '20': 'поэтические переводы',
    '03': 'другие'
}

In [11]:
def get_poem(href):
    response = requests.get(href)
    soup = BeautifulSoup(response.content)
    
    # page don't exists
#     if not soup.body.find("div", id="container"):
#         if not_found:
#             print('2 pages are not found')
#             break
#         not_found = True
#         continue

    check_author = \
    soup.body.find("div", id="container").find("div", {"class": "maintext"}).find("div", {"class": "titleauthor"})
    check_poem = \
    soup.body.find("div", id="container").find("div", {"class": "maintext"}).find("div", {"class": "text"})

    # https://stihi.ru/2009/01/01/3022 - не отображается
    # https://stihi.ru/2017/01/01/974 - автор скрыл
    if not (check_author or check_poem):
        return None, None

    return check_author.get_text(), check_poem.get_text()

In [12]:
csv_file = open('stihi3.csv', mode='w', encoding='utf-8', newline='')
writer = csv.DictWriter(csv_file, fieldnames=['topic', 'date', 'author', 'text'])
writer.writeheader()

pool = ThreadPool(processes=8)

num_skip = 0
pbar = tqdm()
for y in range(2008, 2023):
    for m in range(1,13):
        for d in range(1,32):
            if (y == 2008 and m == 1):
                continue
            for topic_id, topic_name in topic_dict.items():
                start = 30
                while True:
                    pbar.set_description(f'{topic_id} {y}-{m}-{d} start-{start}')
                    href = f'https://stihi.ru/poems/list.html?topic={topic_id}&year={y}&month={m:0>2}&day={d}&start={start}'
                    response = requests.get(href)
                    soup = BeautifulSoup(response.content)

                    url_lists = soup.body.find("div", id="container") \
                                         .find("div", {"class": "maintext"}) \
                                         .find("index") \
                                         .find_all("ul", {"type": "square"}, recursive=False)

                    data_ids = ["https://stihi.ru" + str(tag.a['href'])
                                for url_list in url_lists for tag in url_list.find_all("li")]
                    if len(data_ids) == 0:
                        break
                    
                    for author, poem in pool.imap_unordered(get_poem, data_ids):
                        pbar.update(1)

                        if author is None:
                            num_skip += 1
                            pbar.set_postfix({'num_skip': num_skip})
                            continue

                        writer.writerow({'topic':topic_name, 'date': f'{y}-{m}-{d}', 'author': author, 'text': poem})
                            
                    start += 30  # go to next page
                        
pbar.close()        
csv_file.close()

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [13]:
pbar.close()
csv_file.close()       

### tries 

In [4]:
topic_dict = {
    '13': 'мистика и эзотерика',
    '22': 'сонеты, канцоны, рондо',
}

In [28]:
response = requests.get("https://stihi.ru/2009/01/01/323492837")
soup = BeautifulSoup(response.content)

In [24]:
if not soup.body.find("div", id="container").find("div", {"class": "maintext"}).find("div", {"class": "titleauthor"}):
    print('jj')

AttributeError: 'NoneType' object has no attribute 'find'

In [16]:
print(soup.body.find("div", id="container").find("div", {"class": "maintext"}).find("div", {"class": "titleauthor"}).contents)
soup.body.find("div", id="container").find("div", {"class": "maintext"}).find("div", {"class": "text"}).contents

AttributeError: 'NoneType' object has no attribute 'contents'

In [2]:
response = requests.get("https://stihi.ru/poems/list.html?topic=01&year=2022&month=04&day=9")
soup = BeautifulSoup(response.content)

In [11]:
url_lists = soup.body.find("div", id="container").find("div", {"class": "maintext"}).find_all("ul", {"type": "square"})
for url_list in url_lists:
    for url_tag in url_list.find_all("li"):
        data_id = url_tag.a['href']
        get_poem("https://stihi.ru" + str(data_id), topic_id)
    break

/2022/04/09/8098
/2022/04/09/8095
/2022/04/09/8094
/2022/04/09/8093
/2022/04/09/8089
/2022/04/09/8087
/2022/04/09/8083
/2022/04/09/8080
/2022/04/09/8077
/2022/04/09/8076
/2022/04/09/8073
/2022/04/09/8072
/2022/04/09/8069
/2022/04/09/8059
/2022/04/09/8055
